In [25]:
from importlib import reload
import audioDatastore.AudioDatastore as myads
reload(myads)
import os.path
from collections import Counter
import random


In [18]:

datasetFolder = r"C:\Users\David\OneDrive\Desktop\matlab\data\speech_commands_v0.01"

ads = myads.AudioDatastore()
ads.populate(datasetFolder,include_sub_folders=True, label_source=True)

In [22]:
ads = myads.subset(ads, label='stop')

speakers = []
for file in ads.files:
    nm = os.path.basename(file)
    nm = nm.split('_')[0]
    speakers.append('a' + nm)

ads.set(labels=speakers)

In [28]:
num_speakers_to_enroll = 10
label_count = Counter(ads.labels)
for_enroll_and_test_set = []
for key, cnts in list(label_count.items()):
    if cnts >= 3:
        for_enroll_and_test_set.append(key)
for_enroll = random.sample(for_enroll_and_test_set, num_speakers_to_enroll)
ads_enroll_and_validate = myads.subset(ads, label=for_enroll)
ads_enroll, _ = myads.split(ads_enroll_and_validate, int(len(ads_enroll_and_validate.labels) / 2))
ads_test = myads.subset(ads, for_enroll_and_test_set)
ads_test = myads.filter(ads_test, ads_enroll.labels)
ads_train_ubm = myads.filter(ads, ads_test.labels)
ads_train_ubm = myads.filter(ads_train_ubm, ads_enroll.labels)